In [ ]:
import os

from dotenv import find_dotenv, load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [ ]:
# * Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

# * Verifica se a API key está configurada
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# * Configura o modelo com parâmetros específicos
model: ChatOpenAI = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.0,  # Controla a criatividade das respostas
)  # type: ignore

In [ ]:
prompt_tutor = ChatPromptTemplate.from_template(
    "Você é um tutor de langchain. Responda a pergunta: {pergunta}"
)

output_parser = StrOutputParser()

# Chain usando LCEL
tutor_langchain = prompt_tutor | model | output_parser

response = tutor_langchain.invoke({"pergunta": "PromptTemplate x ChatPromptTemplate"})
print(response)

Claro! Vamos entender a diferença entre **PromptTemplate** e **ChatPromptTemplate** no LangChain:

---

## **PromptTemplate**

- **O que é?**  
  É uma classe usada para criar prompts de texto simples, geralmente para modelos de linguagem tradicionais (como GPT-3, GPT-4 em modo completions).
- **Como funciona?**  
  Você define um template de string com variáveis, e depois preenche essas variáveis com valores específicos.
- **Exemplo:**
  ```python
  from langchain.prompts import PromptTemplate

  prompt = PromptTemplate(
      input_variables=["produto"],
      template="Descreva as vantagens do produto {produto}."
  )

  print(prompt.format(produto="notebook"))
  # Saída: Descreva as vantagens do produto notebook.
  ```

- **Quando usar?**  
  Quando você quer gerar um texto simples, sem estrutura de conversa (mensagens de usuário, assistente, sistema).

---

## **ChatPromptTemplate**

- **O que é?**  
  É uma classe para criar prompts estruturados como conversas, com múltiplas mensa

In [ ]:
# Template com múltiplas variávies
prompt_profissao = ChatPromptTemplate.from_template("""
    Você é um {profissao} especializado em {assunto}.
    Responda a seguinte pergunta de forma {estilo}:

    Pergunta: {question}
""")
parser = StrOutputParser()
chain_profissao = prompt_profissao | model | parser

response = chain_profissao.invoke(
    {
        "profissao": "Professor",
        "assunto": "Langchain",
        "estilo": "didática e simples",
        "question": "Como funciona multi workflow com chain?",
    }
)
print(response)

Claro! Vamos entender de forma simples:

**O que é um "workflow" no Langchain?**  
Um workflow é uma sequência de passos (ou tarefas) que você quer que o seu programa execute. No Langchain, cada passo pode ser uma "chain" (cadeia), que é como um bloco de processamento de informações.

**O que é "multi workflow com chain"?**  
É quando você monta vários fluxos de trabalho (workflows) diferentes, cada um com suas próprias chains, e faz com que eles funcionem juntos ou em paralelo.

### Como funciona na prática?

1. **Chains individuais:**  
   Cada chain faz uma tarefa específica. Por exemplo:
   - Uma chain para buscar informações.
   - Outra chain para resumir textos.
   - Outra para traduzir.

2. **Combinando chains:**  
   Você pode juntar várias chains em um workflow maior. Por exemplo:
   - Primeiro busca a informação (chain 1).
   - Depois resume (chain 2).
   - Depois traduz (chain 3).

3. **Multi workflow:**  
   Você pode ter vários desses fluxos rodando ao mesmo tempo ou escol

In [ ]:
logs = []
parser = StrOutputParser()
parser_and_log_output_chain = RunnableParallel(
    output=parser, log=RunnableLambda(lambda x: logs.append(x))
)

In [ ]:
idea_prompt = PromptTemplate(
    template=(
        """
    Você um consultor de negócios online creativo.
    Crie uma ideia inovadora para um novo negócio da industria: {industria}.
    Forneça uma descrição concisa da ideia.
    """
    )
)

In [ ]:
idea_chain = idea_prompt | model | parser_and_log_output_chain

In [ ]:
idea_response = idea_chain.invoke("Agentes de IA-produção de café")

In [ ]:
idea_response["output"]

'**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**\n\n**Descrição Concisa:**  \nA CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.'

In [ ]:
logs

[AIMessage(content='**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**\n\n**Descrição Concisa:**  \nA CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'complet

In [ ]:
for message in logs:
    print(message.content)

**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**

**Descrição Concisa:**  
A CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.


In [ ]:
analysis_prompt = PromptTemplate(
    template="""
    Analise a seguinte ideia de negócio: {idea}.
    Identifique 3 pontos fortes e 3 potenciais problemas na ideia.
    """
)

In [ ]:
analysis_chain = analysis_prompt | model | parser_and_log_output_chain

In [ ]:
analysis_result = analysis_chain.invoke(idea_response["output"])

In [ ]:
logs

[AIMessage(content='**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**\n\n**Descrição Concisa:**  \nA CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'complet

In [ ]:
analysis_result["output"]

'**Pontos Fortes:**\n\n1. **Automação e Eficiência Operacional:**  \n   A proposta de usar agentes de IA para gerenciar estoque, automação da torra e preparo, além do atendimento ao cliente, pode reduzir custos operacionais, minimizar erros humanos e otimizar processos, tornando a cafeteria mais eficiente e lucrativa.\n\n2. **Personalização da Experiência do Cliente:**  \n   O uso de IA para recomendações personalizadas e interação via chatbot pode aumentar a satisfação e fidelização dos clientes, criando uma experiência diferenciada e inovadora em relação à concorrência.\n\n3. **Redução de Desperdícios e Sustentabilidade:**  \n   O monitoramento em tempo real e a previsão de tendências de consumo ajudam a evitar excessos de estoque e desperdício de grãos, contribuindo para uma operação mais sustentável e com menor impacto ambiental.\n\n---\n\n**Potenciais Problemas:**\n\n1. **Alto Custo de Implementação Inicial:**  \n   A integração de sensores IoT, sistemas de IA e automação pode exi

In [ ]:
for message in logs:
    print(message.content)

**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**

**Descrição Concisa:**  
A CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.
**Pontos Fortes:**

1. **Automação e Eficiência Operacional:**  
   A proposta de usar agentes de IA para

In [ ]:
report_prompt = PromptTemplate(
    template="""
    Aqui é a análise de negócio:
    Destaque os pontos fortes e fracos: {output}
    Crie um relatório de negócio estruturado.
    """
)

In [ ]:
class RelatorioAnalise(BaseModel):
    """Pontos fortes e fracos da nova ideia de négocio"""

    pontos_fortes: list = Field(
        default=[], description="Lista dos pontos fortes do negócio"
    )
    pontos_fracos: list = Field(default=[], description="lista dos pontos fracos")

In [ ]:
report_chain = report_prompt | model.with_structured_output(
    RelatorioAnalise, method="function_calling"
)

In [ ]:
report_result = report_chain.invoke(analysis_result["output"])

In [ ]:
report_result

RelatorioAnalise(pontos_fortes=['Automação e Eficiência Operacional: Redução de custos, minimização de erros humanos e otimização de processos por meio de agentes de IA.', 'Personalização da Experiência do Cliente: Recomendações personalizadas e interação via chatbot aumentam satisfação e fidelização.', 'Redução de Desperdícios e Sustentabilidade: Monitoramento em tempo real e previsão de consumo evitam excessos de estoque e desperdício, promovendo sustentabilidade.'], pontos_fracos=['Alto Custo de Implementação Inicial: Investimento elevado em sensores IoT, IA e automação pode ser impeditivo para pequenos negócios.', 'Resistência Cultural e de Mercado: Possível resistência de proprietários, funcionários e clientes à adoção de tecnologia em um ambiente tradicional.', 'Complexidade Técnica e Manutenção: Necessidade de conhecimento técnico especializado e riscos operacionais em caso de falhas ou falta de suporte.'])

In [ ]:
logs

[AIMessage(content='**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**\n\n**Descrição Concisa:**  \nA CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'complet

In [ ]:
for message in logs:
    print(message.content)

**Ideia de Negócio: CoffeeAI – Agentes de IA para Cafeterias Inteligentes**

**Descrição Concisa:**  
A CoffeeAI oferece uma plataforma de agentes de inteligência artificial especializados na automação e otimização de cafeterias e torrefações. Os agentes de IA gerenciam todo o ciclo de produção do café: desde o controle inteligente do estoque de grãos, passando pela automação do processo de torra e preparo, até o atendimento personalizado ao cliente (recomendações de blends, sugestões de métodos de preparo e interação via chatbot). A solução integra sensores IoT para monitoramento em tempo real da qualidade do café e utiliza machine learning para prever tendências de consumo, reduzindo desperdícios e aumentando a satisfação do cliente. Cafeterias podem, assim, operar de forma mais eficiente, sustentável e inovadora, destacando-se no mercado com experiências únicas e personalizadas.
**Pontos Fortes:**

1. **Automação e Eficiência Operacional:**  
   A proposta de usar agentes de IA para

In [ ]:
report_result.pontos_fortes

['Automação e Eficiência Operacional: Redução de custos, minimização de erros humanos e otimização de processos por meio de agentes de IA.',
 'Personalização da Experiência do Cliente: Recomendações personalizadas e interação via chatbot aumentam satisfação e fidelização.',
 'Redução de Desperdícios e Sustentabilidade: Monitoramento em tempo real e previsão de consumo evitam excessos de estoque e desperdício, promovendo sustentabilidade.']

In [ ]:
report_result.pontos_fracos

['Alto Custo de Implementação Inicial: Investimento elevado em sensores IoT, IA e automação pode ser impeditivo para pequenos negócios.',
 'Resistência Cultural e de Mercado: Possível resistência de proprietários, funcionários e clientes à adoção de tecnologia em um ambiente tradicional.',
 'Complexidade Técnica e Manutenção: Necessidade de conhecimento técnico especializado e riscos operacionais em caso de falhas ou falta de suporte.']

In [ ]:
A2A_chain = (
    RunnablePassthrough()
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [ ]:
A2A_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [ ]:
A2A_result = A2A_chain.invoke("restaurante")

In [ ]:
A2A_result

RelatorioAnalise(pontos_fortes=['Experiência exclusiva e inovadora, diferenciando-se dos restaurantes tradicionais.', 'Alcance de novos públicos e mercados ao explorar diferentes bairros e parcerias.', 'Alto engajamento nas redes sociais, criando comunidade e divulgação orgânica.'], pontos_fracos=['Logística complexa e custos operacionais elevados devido à itinerância.', 'Dificuldade em garantir consistência e qualidade em cada edição.', 'Risco de baixa previsibilidade de demanda, podendo gerar baixa ocupação ou desperdício.'])